In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

<b> Loading and Transforming data </b>

In [3]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_data = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 182040794/182040794 [00:03<00:00, 46540761.98it/s]


100%|██████████| 64275384/64275384 [00:01<00:00, 37696392.43it/s]


<b>Defining LeNet5 Architecture </b>

In [4]:
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

<b> Defining AlexNet Architecture </b>

In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = models.alexnet(pretrained=False).features
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

<b> Loading Pretrained Weights </b>

In [6]:
models_list = {
    "LeNet-5": LeNet5,
    "AlexNet": AlexNet,
    "VGG": models.vgg16,
    "ResNet-18": models.resnet18,
    "ResNet-50": models.resnet50,
    "ResNet-101": models.resnet101
}

def load_pretrained_weights(model_name):
    if model_name == "LeNet-5":
        return LeNet5().to(device)
    elif model_name == "AlexNet":
        model = AlexNet(num_classes=10)
        pretrained_dict = models.alexnet(pretrained=True).state_dict()
        model_dict = model.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # Update the state dict
        model_dict.update(pretrained_dict)
        # Load the updated state dict
        model.load_state_dict(model_dict)
        return model.to(device)
    else:
        model = models_list[model_name](pretrained=True)
        # Replace the last layer with custom output layer to match SVHN
        if "ResNet" in model_name:
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 10)
        else:
            num_ftrs = model.classifier[-1].in_features
            model.classifier[-1] = nn.Linear(num_ftrs, 10)
        return model.to(device)

In [7]:
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {100*train_accuracy:.2f}%")


In [8]:
def test_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_accuracy = correct / total
    print(f"Test Accuracy: {100*test_accuracy:.2f}%")

In [9]:
my_list = []
for model_name in models_list.keys():
    my_list.append(model_name)


<b>LeNet-5 Model </b>

In [10]:
model = load_pretrained_weights(my_list[0])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[0]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[0]}...")
test_model(model)
print()

Training LeNet-5...
Epoch 1/5, Train Loss: 2.2541, Train Accuracy: 18.17%
Epoch 2/5, Train Loss: 2.2358, Train Accuracy: 18.92%
Epoch 3/5, Train Loss: 2.2338, Train Accuracy: 18.92%
Epoch 4/5, Train Loss: 2.2252, Train Accuracy: 18.92%
Epoch 5/5, Train Loss: 2.0072, Train Accuracy: 29.15%
Testing LeNet-5...
Test Accuracy: 47.68%



<b>VGG Model</b>

In [14]:
model = load_pretrained_weights(my_list[2])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[2]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[2]}...")
test_model(model)
print()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 58.0MB/s]


Training VGG...
Epoch 1/5, Train Loss: 0.5869, Train Accuracy: 80.89%
Epoch 2/5, Train Loss: 0.2762, Train Accuracy: 91.81%
Epoch 3/5, Train Loss: 0.2141, Train Accuracy: 93.77%
Epoch 4/5, Train Loss: 0.1776, Train Accuracy: 94.92%
Epoch 5/5, Train Loss: 0.1503, Train Accuracy: 95.75%
Testing VGG...
Test Accuracy: 94.52%



<b>AlexNet Model</b>

In [ ]:
model = load_pretrained_weights(my_list[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[1]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[1]}...")
test_model(model)
print()

<b>Resnet 18</b>

In [11]:
model = load_pretrained_weights(my_list[3])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[3]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[3]}...")
test_model(model)
print()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 154MB/s]


Training ResNet-18...
Epoch 1/5, Train Loss: 0.7883, Train Accuracy: 73.97%
Epoch 2/5, Train Loss: 0.3403, Train Accuracy: 89.32%
Epoch 3/5, Train Loss: 0.2563, Train Accuracy: 92.12%
Epoch 4/5, Train Loss: 0.2086, Train Accuracy: 93.48%
Epoch 5/5, Train Loss: 0.1710, Train Accuracy: 94.65%
Testing ResNet-18...
Test Accuracy: 92.24%



<b>Resnet 50</b>

In [12]:
model = load_pretrained_weights(my_list[4])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[4]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[4]}...")
test_model(model)
print()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 167MB/s]


Training ResNet-50...
Epoch 1/5, Train Loss: 0.7058, Train Accuracy: 76.58%
Epoch 2/5, Train Loss: 0.2708, Train Accuracy: 91.82%
Epoch 3/5, Train Loss: 0.1907, Train Accuracy: 94.26%
Epoch 4/5, Train Loss: 0.1399, Train Accuracy: 95.68%
Epoch 5/5, Train Loss: 0.1026, Train Accuracy: 96.85%
Testing ResNet-50...
Test Accuracy: 93.11%



<b>Resnet 100</b>

In [13]:
model = load_pretrained_weights(my_list[5])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(f"Training {my_list[5]}...")
train_model(model, criterion, optimizer)
print(f"Testing {my_list[5]}...")
test_model(model)
print()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 138MB/s]


Training ResNet-101...
Epoch 1/5, Train Loss: 0.7050, Train Accuracy: 76.53%
Epoch 2/5, Train Loss: 0.2707, Train Accuracy: 91.75%
Epoch 3/5, Train Loss: 0.1840, Train Accuracy: 94.39%
Epoch 4/5, Train Loss: 0.1293, Train Accuracy: 96.01%
Epoch 5/5, Train Loss: 0.0935, Train Accuracy: 97.15%
Testing ResNet-101...
Test Accuracy: 92.47%

